In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# 데이터 경로 설정
damaged_path = 'car-damage-dataset/data1a/training/00-damage'
undamaged_path = 'car-damage-dataset/data1a/training/01-whole'

# 데이터 로드 및 라벨링
def load_data_from_folders(damaged_path, undamaged_path):
    images = []
    labels = []
    
    for img_name in os.listdir(damaged_path):
        img_path = os.path.join(damaged_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # 크기 조정
            images.append(img)
            labels.append(1)  # '파괴' 라벨

    for img_name in os.listdir(undamaged_path):
        img_path = os.path.join(undamaged_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(0)  # '비파괴' 라벨
    
    return np.array(images), np.array(labels)

X, y = load_data_from_folders(damaged_path, undamaged_path)
X = X / 255.0  # 정규화
y = to_categorical(y)  # 원-핫 인코딩

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # 출력층
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")
